In [1]:
import pandas as pd
import numpy as np
import glob, os
import re

# Source

#### basketball-reference.com
https://www.basketball-reference.com/teams/HOU/2017_games.html

- saved regular and playoff seasons from 2008 - 2017 to csv

In [136]:
ls sports_data/NBA/

hou_rockets_plo_08.csv*  hou_rockets_plo_17.csv*  hou_rockets_reg_13.csv*
hou_rockets_plo_09.csv*  hou_rockets_reg_08.csv*  hou_rockets_reg_14.csv*
hou_rockets_plo_13.csv*  hou_rockets_reg_09.csv*  hou_rockets_reg_15.csv*
hou_rockets_plo_14.csv*  hou_rockets_reg_10.csv*  hou_rockets_reg_16.csv*
hou_rockets_plo_15.csv*  hou_rockets_reg_11.csv*  hou_rockets_reg_17.csv*
hou_rockets_plo_16.csv*  hou_rockets_reg_12.csv*


In [81]:
path = 'sports_data/NBA'
all_files = glob.glob(os.path.join(path, "*.csv")) 

df_from_each_file = (pd.read_csv(f) for f in all_files)
df   = pd.concat(df_from_each_file, ignore_index=True)

In [82]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 837 entries, 0 to 836
Data columns (total 15 columns):
G             837 non-null int64
Date          837 non-null object
Unnamed: 2    837 non-null object
Unnamed: 3    8 non-null object
Unnamed: 4    837 non-null object
Unnamed: 5    418 non-null object
Opponent      837 non-null object
Unnamed: 7    837 non-null object
Unnamed: 8    52 non-null object
Tm            837 non-null int64
Opp           837 non-null int64
W             837 non-null int64
L             837 non-null int64
Streak        837 non-null object
Notes         1 non-null object
dtypes: int64(5), object(10)
memory usage: 98.2+ KB


In [83]:
df.head()

,G,Date,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Opponent,Unnamed: 7,Unnamed: 8,Tm,Opp,W,L,Streak,Notes
0,1,"Sat, Apr 18, 2015",9:30p ET,NaN,Box Score,NaN,Dallas Mavericks,W,NaN,118,108,1,0,W 1,NaN
1,2,"Tue, Apr 21, 2015",9:30p ET,NaN,Box Score,NaN,Dallas Mavericks,W,NaN,111,99,2,0,W 2,NaN
2,3,"Fri, Apr 24, 2015",7:00p ET,NaN,Box Score,@,Dallas Mavericks,W,NaN,130,128,3,0,W 3,NaN
3,4,"Sun, Apr 26, 2015",9:00p ET,NaN,Box Score,@,Dallas Mavericks,L,NaN,109,121,3,1,L 1,NaN
4,5,"Tue, Apr 28, 2015",8:00p ET,NaN,Box Score,NaN,Dallas Mavericks,W,NaN,103,94,4,1,W 1,NaN


## Create slice with only needed columns

In [84]:
hou_rockets = df[['Date','Unnamed: 2','Unnamed: 5','Opponent','Unnamed: 7','Tm','Opp']]

## Create a column with Houston Rockets

In [85]:
hou_rockets['team'] ='Houston Rockets'

/home/salas/miniconda3/envs/springboard/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [86]:
hou_rockets = hou_rockets[['Date','Unnamed: 2', 'team','Tm','Unnamed: 5','Opponent','Opp','Unnamed: 7']]

In [87]:
hou_rockets.head()

,Date,Unnamed: 2,team,Tm,Unnamed: 5,Opponent,Opp,Unnamed: 7
0,"Sat, Apr 18, 2015",9:30p ET,Houston Rockets,118,NaN,Dallas Mavericks,108,W
1,"Tue, Apr 21, 2015",9:30p ET,Houston Rockets,111,NaN,Dallas Mavericks,99,W
2,"Fri, Apr 24, 2015",7:00p ET,Houston Rockets,130,@,Dallas Mavericks,128,W
3,"Sun, Apr 26, 2015",9:00p ET,Houston Rockets,109,@,Dallas Mavericks,121,L
4,"Tue, Apr 28, 2015",8:00p ET,Houston Rockets,103,NaN,Dallas Mavericks,94,W


## Rename Columns

In [88]:
hou_rockets.rename(columns={
    'Unnamed: 2':'Time',
    'team': 'Team',
    'Tm': 'Team_score',
    'Unnamed: 5': 'Location',
    'Opp': 'Opponent_score',
    'Unnamed: 7': 'Result'
},inplace=True)

In [89]:
hou_rockets.head()

,Date,Time,Team,Team_score,Location,Opponent,Opponent_score,Result
0,"Sat, Apr 18, 2015",9:30p ET,Houston Rockets,118,NaN,Dallas Mavericks,108,W
1,"Tue, Apr 21, 2015",9:30p ET,Houston Rockets,111,NaN,Dallas Mavericks,99,W
2,"Fri, Apr 24, 2015",7:00p ET,Houston Rockets,130,@,Dallas Mavericks,128,W
3,"Sun, Apr 26, 2015",9:00p ET,Houston Rockets,109,@,Dallas Mavericks,121,L
4,"Tue, Apr 28, 2015",8:00p ET,Houston Rockets,103,NaN,Dallas Mavericks,94,W


## Inspect Data

In [90]:
hou_rockets.Date.unique()  # need split day

array(['Sat, Apr 18, 2015', 'Tue, Apr 21, 2015', 'Fri, Apr 24, 2015',
       'Sun, Apr 26, 2015', 'Tue, Apr 28, 2015', 'Mon, May 4, 2015',
       'Wed, May 6, 2015', 'Fri, May 8, 2015', 'Sun, May 10, 2015',
       'Tue, May 12, 2015', 'Thu, May 14, 2015', 'Sun, May 17, 2015',
       'Tue, May 19, 2015', 'Thu, May 21, 2015', 'Sat, May 23, 2015',
       'Mon, May 25, 2015', 'Wed, May 27, 2015', 'Sat, Apr 16, 2016',
       'Mon, Apr 18, 2016', 'Thu, Apr 21, 2016', 'Sun, Apr 24, 2016',
       'Wed, Apr 27, 2016', 'Sun, Apr 16, 2017', 'Wed, Apr 19, 2017',
       'Fri, Apr 21, 2017', 'Sun, Apr 23, 2017', 'Tue, Apr 25, 2017',
       'Mon, May 1, 2017', 'Wed, May 3, 2017', 'Fri, May 5, 2017',
       'Sun, May 7, 2017', 'Tue, May 9, 2017', 'Thu, May 11, 2017',
       'Tue, Oct 30, 2007', 'Thu, Nov 1, 2007', 'Sat, Nov 3, 2007',
       'Mon, Nov 5, 2007', 'Tue, Nov 6, 2007', 'Fri, Nov 9, 2007',
       'Sun, Nov 11, 2007', 'Tue, Nov 13, 2007', 'Wed, Nov 14, 2007',
       'Fri, Nov 16, 2007', 'Sat,

In [91]:
hou_rockets.Time.unique()  # need correct time

array(['9:30p ET', '7:00p ET', '9:00p ET', '8:00p ET', '10:30p ET',
       '8:30p ET', '3:30p ET', '7:30p ET', '6:00p ET', '6:30p ET',
       '12:30p ET', '1:30p ET', '2:30p ET', '12:00p ET', '1:00p ET',
       '4:00p ET', '2:00p ET', '3:00p ET', '10:00p ET', '5:30p ET',
       '5:00p ET'], dtype=object)

In [92]:
hou_rockets.Team.unique()  # ok

array(['Houston Rockets'], dtype=object)

In [93]:
hou_rockets.Team_score.unique()  # ok

array([118, 111, 130, 109, 103, 101, 115,  99,  95, 124, 119, 113, 106,
        98,  80, 128,  90,  78,  97,  94,  81, 105, 126,  96,  92, 125,
       107,  75,  89, 104,  85,  84,  91,  88, 100,  83, 116,  82,  93,
       108, 112, 110, 117,  74,  69,  79,  86, 131, 114, 120,  87,  73,
       102,  66, 127, 121, 129, 123, 140, 122, 136, 134, 145,  77, 132,
       137, 133, 142, 139, 135])

In [94]:
hou_rockets.Location.unique()  # 

array([nan, '@'], dtype=object)

In [95]:
hou_rockets.Opponent.unique()  # ok

array(['Dallas Mavericks', 'Los Angeles Clippers',
       'Golden State Warriors', 'Oklahoma City Thunder',
       'San Antonio Spurs', 'Los Angeles Lakers', 'Utah Jazz',
       'Portland Trail Blazers', 'Milwaukee Bucks', 'Charlotte Bobcats',
       'Memphis Grizzlies', 'Phoenix Suns', 'Miami Heat',
       'Denver Nuggets', 'Sacramento Kings', 'New Jersey Nets',
       'Toronto Raptors', 'Philadelphia 76ers', 'Detroit Pistons',
       'Orlando Magic', 'Chicago Bulls', 'Boston Celtics',
       'New York Knicks', 'Washington Wizards', 'Minnesota Timberwolves',
       'New Orleans Hornets', 'Seattle SuperSonics', 'Indiana Pacers',
       'Cleveland Cavaliers', 'Atlanta Hawks', 'Brooklyn Nets',
       'New Orleans Pelicans', 'Charlotte Hornets'], dtype=object)

In [96]:
hou_rockets.Opponent.value_counts(dropna=False)  # ok

Golden State Warriors     46
San Antonio Spurs         46
Dallas Mavericks          44
Los Angeles Clippers      42
Memphis Grizzlies         40
Oklahoma City Thunder     38
Denver Nuggets            37
Minnesota Timberwolves    36
Sacramento Kings          36
Phoenix Suns              36
Los Angeles Lakers        35
Portland Trail Blazers    35
Utah Jazz                 35
New Orleans Hornets       23
Toronto Raptors           20
Washington Wizards        20
Boston Celtics            19
Cleveland Cavaliers       19
Miami Heat                19
Atlanta Hawks             19
Milwaukee Bucks           19
Chicago Bulls             19
Detroit Pistons           19
New York Knicks           19
Indiana Pacers            19
Orlando Magic             19
Philadelphia 76ers        19
New Orleans Pelicans      16
Charlotte Bobcats         14
Brooklyn Nets             10
New Jersey Nets            9
Charlotte Hornets          6
Seattle SuperSonics        4
Name: Opponent, dtype: int64

In [97]:
hou_rockets.Opponent_score.unique()  # ok

array([108,  99, 128, 121,  94, 117, 109, 124, 103, 107, 100, 110, 115,
       104,  96, 114,  87, 111,  93,  95,  80,  81,  88,  82, 105,  90,
        98,  71, 113,  92,  89,  77,  97, 112,  83,  79,  91,  84,  86,
        85,  69,  73,  75, 106, 122,  66, 102, 101,  76,  78,  74, 130,
       118, 125, 133, 123, 119, 116, 132,  70, 120, 134, 137,  65, 131,
       126, 140, 127])

In [98]:
hou_rockets.Result.unique()  # ok

array(['W', 'L'], dtype=object)

## Location Column

- change @ to away, nan to home

In [99]:
hou_rockets.Location.value_counts(dropna=False)

NaN    419
@      418
Name: Location, dtype: int64

In [101]:
hou_rockets['Location'] = np.where(hou_rockets['Location'] =='@', 'away', 'home')

In [102]:
hou_rockets.Location.value_counts(dropna=False)

home    837
Name: Location, dtype: int64

In [103]:
hou_rockets.head()

,Date,Time,Team,Team_score,Location,Opponent,Opponent_score,Result
0,"Sat, Apr 18, 2015",9:30p ET,Houston Rockets,118,home,Dallas Mavericks,108,W
1,"Tue, Apr 21, 2015",9:30p ET,Houston Rockets,111,home,Dallas Mavericks,99,W
2,"Fri, Apr 24, 2015",7:00p ET,Houston Rockets,130,home,Dallas Mavericks,128,W
3,"Sun, Apr 26, 2015",9:00p ET,Houston Rockets,109,home,Dallas Mavericks,121,L
4,"Tue, Apr 28, 2015",8:00p ET,Houston Rockets,103,home,Dallas Mavericks,94,W


## Date column

- split Date:
    - Day : Sun
    - Date : Month, day, Year

In [122]:
# hou_rockets['Day'] = hou_rockets['Date'].str.split(',').apply(lambda x: x[0])
# hou_rockets['Date'] = hou_rockets['Date'].str.split(',').apply(lambda x: x[1:])

In [105]:
hou_rockets.head()

,Date,Time,Team,Team_score,Location,Opponent,Opponent_score,Result
0,"Sat, Apr 18, 2015",9:30p ET,Houston Rockets,118,home,Dallas Mavericks,108,W
1,"Tue, Apr 21, 2015",9:30p ET,Houston Rockets,111,home,Dallas Mavericks,99,W
2,"Fri, Apr 24, 2015",7:00p ET,Houston Rockets,130,home,Dallas Mavericks,128,W
3,"Sun, Apr 26, 2015",9:00p ET,Houston Rockets,109,home,Dallas Mavericks,121,L
4,"Tue, Apr 28, 2015",8:00p ET,Houston Rockets,103,home,Dallas Mavericks,94,W


In [120]:
#day = hou_rockets['Date'].str.split(',').apply(lambda x: x[0])

In [118]:
hou_rockets['Date'] = pd.to_datetime(hou_rockets['Date'])

In [123]:
hou_rockets.head()

,Date,Time,Team,Team_score,Location,Opponent,Opponent_score,Result
0,2015-04-18,9:30p ET,Houston Rockets,118,home,Dallas Mavericks,108,W
1,2015-04-21,9:30p ET,Houston Rockets,111,home,Dallas Mavericks,99,W
2,2015-04-24,7:00p ET,Houston Rockets,130,home,Dallas Mavericks,128,W
3,2015-04-26,9:00p ET,Houston Rockets,109,home,Dallas Mavericks,121,L
4,2015-04-28,8:00p ET,Houston Rockets,103,home,Dallas Mavericks,94,W


In [124]:
hou_rockets = hou_rockets.set_index('Date').sort_index(ascending=True)

In [126]:
hou_rockets.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 837 entries, 2007-10-30 to 2017-05-11
Data columns (total 7 columns):
Time              837 non-null object
Team              837 non-null object
Team_score        837 non-null int64
Location          837 non-null object
Opponent          837 non-null object
Opponent_score    837 non-null int64
Result            837 non-null object
dtypes: int64(2), object(5)
memory usage: 52.3+ KB


In [129]:
hou_rockets.head()

,Time,Team,Team_score,Location,Opponent,Opponent_score,Result
Date,,,,,,,
2007-10-30,7:30p ET,Houston Rockets,95,home,Los Angeles Lakers,93,W
2007-11-01,7:00p ET,Houston Rockets,106,home,Utah Jazz,95,W
2007-11-03,7:30p ET,Houston Rockets,89,home,Portland Trail Blazers,80,W
2007-11-05,7:30p ET,Houston Rockets,98,home,Dallas Mavericks,107,L
2007-11-06,7:30p ET,Houston Rockets,89,home,San Antonio Spurs,81,W


## Save clean data to  to csv

In [130]:
hou_rockets.to_csv('clean_data/hou_rockets_08_17_clean.csv')

In [131]:
ls clean_data/

astros_08_17_clean.csv*       rice_08_17_clean.csv*  uh_08_17_clean.csv*
hou_rockets_08_17_clean.csv*  tsu_08_17_clean.csv*


In [132]:
## Finish